In [1]:
# Add user specific python libraries to path
import sys
sys.path.insert(0, "/home/smehra/local-packages")

In [2]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(dashboard_address = 'localhost:7920', 
                       n_workers = 16, 
                       processes = True, 
                       threads_per_worker = 16,
                       memory_limit = '16GB', 
                       local_directory = "/data/tmp/smehra/tmp/dask-worker-space")
client = Client(cluster)


import multiprocessing as mp
from multiprocessing.pool import ThreadPool

import numpy as np
import pandas as pd
import geopandas as gpd
import dask.dataframe as ddf

# enable automated generational garbage collection
import gc
gc.enable()

import time, os
from datetime import timedelta  
from datetime import date
from datetime import datetime


In [3]:
import logging

# setup logging to a specified file
log_file = '/data/tmp/smehra/logs/migration_metrics_computation.log'
logging.basicConfig(filename=log_file,
                            filemode='a+',
                            format='%(asctime)s %(levelname)s %(message)s',
                            datefmt='%Y-%m-%d %H:%M:%S',
                            level=logging.INFO)
logger=logging.getLogger(__name__)


In [4]:
def get_total_migrated_after_time_delta(origin_district, impact_day, impacted_users, total_impacted, time_delta):

    if(((impact_day + time_delta) > 2770) or (total_impacted == 0)):
        
        empty_df = pd.DataFrame(np.NaN, columns=["migrated"], index=district_ids)
        return empty_df.to_dict()['migrated']
    
    users_in_other_districts_after_time_delta = users_per_district_day[(users_per_district_day.day_series == (impact_day + time_delta))].copy()

    users_in_other_districts_after_time_delta['migrated'] = users_in_other_districts_after_time_delta.active_users.apply(lambda migrated: len(impacted_users.intersection(migrated)))
    
    users_in_other_districts_after_time_delta.set_index('district_id', inplace = True)
    
    return users_in_other_districts_after_time_delta[['migrated']].to_dict()['migrated']
    

def get_migration_metrics(district_day):
    
    origin_district = district_day[0]
    impact_day = district_day[1]
    
    logger.info('origin_district: ' + str(origin_district)  + '. impact_day: ' + str(impact_day))
    
    impacted_users = users_per_district_day[(users_per_district_day.district_id == origin_district) & 
                                            (users_per_district_day.day_series == impact_day)].active_users.item()
    total_impacted = len(impacted_users)

    summary = get_total_migrated_after_time_delta(origin_district, impact_day, impacted_users, total_impacted, time_delta)
    
    summary['origin_district'] = origin_district
    summary['impact_day'] = impact_day
    summary['impacted'] = total_impacted
    summary['visit_day'] = impact_day + time_delta

    return summary


# get a list of day_series values
days = pd.DataFrame({'day_series': np.arange(1, 2771, 1).tolist()}) 

# get a list of district ids
district_ids = sorted(gpd.read_file('/data/afg_anon/ShapeFiles/AFG_district_398/district398.shp').DISTID.tolist())
districts = pd.DataFrame({'district_id': district_ids}) 

# get all district - day_series pairs in a dataframe
district_day_all_permutations_df = districts.assign(dummy_col = 1).merge(days.assign(dummy_col = 1), how = 'outer').drop(columns = ["dummy_col"])
district_day_all_permutations_list = district_day_all_permutations_df.to_numpy().tolist()


## Compute visits per district day at user id level

In [5]:
%%time

user_location_directory = '/data/afg_anon/displacement_metrics/home_locations/daily_modal_voice_only_2013-2020_version/'

for fname in os.listdir(user_location_directory):
    
    print(str(datetime.now()) + ' reading ' + fname)
    logger.info('reading ' + fname)
    
    users_per_district_day = pd.read_csv(user_location_directory + fname,
                                         dtype = {'phoneHash1': object,
                                                  'home_location': 'int16',
                                                  'day': object})
    if(len(users_per_district_day) == 0):
        print(str(datetime.now()) + ' Empty dataset. Skipped.')
        logger.info('Empty dataset. Skipped.')
        continue
    
    # get "set" of users for each district each day
    users_per_district_day = users_per_district_day.groupby(['home_location', 'day'])['phoneHash1'].apply(set)
    users_per_district_day = users_per_district_day.reset_index()
    
    # convert dates to day series
    users_per_district_day['day'] = pd.to_datetime(users_per_district_day.day, format='%Y-%m-%d', errors='coerce')
    users_per_district_day['day_series'] = users_per_district_day['day'].dt.date.apply(lambda d: (d - date(2013,3,31)).days)

    # rename and reorder columns
    users_per_district_day.rename(columns = {'home_location': 'district_id', 'phoneHash1': 'active_users'}, inplace = True)
    users_per_district_day = users_per_district_day[['day_series', 'district_id', 'active_users']]
    
    # ensure we have a row for all permutations of district - day.
    users_per_district_day = district_day_all_permutations_df.merge(users_per_district_day, on = ['district_id', 'day_series'], how = 'left')

    # convert no users to empty set
    users_per_district_day['active_users'] = users_per_district_day.active_users.apply(lambda u: set() if u != u else u)
    
    for time_delta in [30]:

        print(str(datetime.now()) + ' Computing for time_delta = ' + str(time_delta))
        logger.info('Computing for time_delta = ' + str(time_delta))
        
        with ThreadPool() as pool:

            # get migration metrics for impacted users of each district - day
            migration_metrics_as_list = pool.map(get_migration_metrics, district_day_all_permutations_list)
        
        # convert migration metrics list to dataframe
        # schema - ['origin_district', 'impact_day', 'impacted', 'visit_day', 101, 102, 103, ... 3401]
        migration_metrics_column_major = pd.DataFrame(migration_metrics_as_list)

        print(str(datetime.now()) + ' Unpivoting dataset.')
        logger.info('Unpivoting dataset.')

        # convert to row major form
        # schema - ['origin_district', 'impact_day', 'impacted', 'visit_day', 'destination_district', 'visits']
        migration_metrics = migration_metrics_column_major.melt(id_vars=['origin_district', 'impact_day', 'impacted', 'visit_day'], var_name='destination_district', value_name='visits')
        migration_metrics = migration_metrics.sort_values(['origin_district', 'impact_day']).reset_index(drop = True)
        migration_metrics = migration_metrics[['origin_district', 'impact_day', 'impacted', 'destination_district', 'visit_day', 'visits']]

        print(str(datetime.now()) + ' Saving dataset.')
        logger.info('Saving dataset.')
        migration_metrics.to_csv('/data/tmp/smehra/aggregated_data/poppy/visits_per_district_day/time_delta_' + str(time_delta) + '_days/user_id_level/' + fname, index = False)



2023-07-09 21:53:03.578549 reading Dx.csv
2023-07-09 21:55:25.687622 Computing for time_delta = 30
2023-07-09 23:19:46.747276 Unpivoting dataset.
2023-07-09 23:21:37.721011 Saving dataset.
2023-07-09 23:36:25.426961 reading 30.csv
2023-07-09 23:38:59.197530 Computing for time_delta = 30
2023-07-10 01:04:58.093453 Unpivoting dataset.
2023-07-10 01:06:52.380615 Saving dataset.
2023-07-10 01:21:49.050655 reading oV.csv
2023-07-10 01:24:22.009866 Computing for time_delta = 30
2023-07-10 02:50:30.394310 Unpivoting dataset.
2023-07-10 02:52:23.743851 Saving dataset.
2023-07-10 03:07:12.506023 reading a1.csv
2023-07-10 03:07:16.633136 Empty dataset. Skipped.
2023-07-10 03:07:16.633505 reading Em.csv
2023-07-10 03:10:35.694984 Computing for time_delta = 30
2023-07-10 04:39:58.147232 Unpivoting dataset.
2023-07-10 04:41:52.601141 Saving dataset.
2023-07-10 04:56:41.093370 reading LN.csv
2023-07-10 04:59:07.039537 Computing for time_delta = 30
2023-07-10 06:25:06.343274 Unpivoting dataset.
2023-

2023-07-11 16:32:30.680701 Computing for time_delta = 30
2023-07-11 18:01:24.015660 Unpivoting dataset.
2023-07-11 18:03:16.124046 Saving dataset.
2023-07-11 18:18:40.130421 reading 7o.csv
2023-07-11 18:22:00.777253 Computing for time_delta = 30
2023-07-11 19:49:49.661875 Unpivoting dataset.
2023-07-11 19:51:40.732285 Saving dataset.
2023-07-11 20:07:11.878739 reading j9.csv
2023-07-11 20:09:42.688513 Computing for time_delta = 30
2023-07-11 21:35:24.177409 Unpivoting dataset.
2023-07-11 21:37:13.516494 Saving dataset.
2023-07-11 21:52:41.410963 reading n5.csv
2023-07-11 21:55:13.392037 Computing for time_delta = 30
2023-07-11 23:20:39.102801 Unpivoting dataset.
2023-07-11 23:22:30.054046 Saving dataset.
2023-07-11 23:37:52.107854 reading j4.csv
2023-07-11 23:37:55.268264 Empty dataset. Skipped.
2023-07-11 23:37:55.269018 reading DZ.csv
2023-07-11 23:41:12.531610 Computing for time_delta = 30
2023-07-12 01:09:00.976372 Unpivoting dataset.
2023-07-12 01:10:51.649586 Saving dataset.
2023

## Convert user id level metrics to impact day level

In [5]:
time_delta = 30
    
visits_dataset_directory = '/data/tmp/smehra/aggregated_data/poppy/visits_per_district_day/time_delta_' + str(time_delta) + '_days/user_id_level/'
filepaths = [visits_dataset_directory + f for f in os.listdir(visits_dataset_directory)]

for impact_day in range(1, 2771):

    print(str(datetime.now()) + ' Computing for impact day ' + str(impact_day))

    user_id_level_datasets = []

    for file in filepaths:

        # read user_id level dataset
        user_id_level_data = ddf.read_csv(file,
                                          dtype = {'origin_district': 'int16',
                                                   'impact_day': 'int16',
                                                   'impacted': 'int32',
                                                   'destination_district': 'int16',
                                                   'visit_day': 'int16',
                                                   'visits': 'float64'})

        # keep only data for a specific impact_day
        user_id_level_data = user_id_level_data[user_id_level_data.impact_day == impact_day]

        # add it to the list of dataframes
        user_id_level_datasets.append(user_id_level_data)

    # concatenate together data for a specific impact_day from all user id datasets
    impact_day_level_data = ddf.concat(user_id_level_datasets)            
    impact_day_level_data.to_csv('/data/tmp/smehra/aggregated_data/poppy/visits_per_district_day/time_delta_' + str(time_delta) + '_days/impact_day_level/impact_day_' + str(impact_day), index = False)



2023-07-13 10:20:46.005286 Computing for impact day 1
2023-07-13 10:24:29.199542 Computing for impact day 2
2023-07-13 10:28:14.018181 Computing for impact day 3
2023-07-13 10:31:58.325206 Computing for impact day 4
2023-07-13 10:35:43.326207 Computing for impact day 5
2023-07-13 10:39:28.659763 Computing for impact day 6
2023-07-13 10:43:14.734391 Computing for impact day 7
2023-07-13 10:46:59.748351 Computing for impact day 8
2023-07-13 10:50:46.350134 Computing for impact day 9
2023-07-13 10:54:34.539916 Computing for impact day 10
2023-07-13 10:58:22.130705 Computing for impact day 11
2023-07-13 11:02:11.683884 Computing for impact day 12
2023-07-13 11:06:04.309984 Computing for impact day 13
2023-07-13 11:10:02.864515 Computing for impact day 14
2023-07-13 11:13:47.902684 Computing for impact day 15
2023-07-13 11:17:32.027442 Computing for impact day 16
2023-07-13 11:21:17.030999 Computing for impact day 17
2023-07-13 11:25:02.398077 Computing for impact day 18
2023-07-13 11:28:45

2023-07-13 19:32:12.927860 Computing for impact day 150
2023-07-13 19:35:54.172304 Computing for impact day 151
2023-07-13 19:39:34.809778 Computing for impact day 152
2023-07-13 19:43:15.361507 Computing for impact day 153
2023-07-13 19:46:56.235860 Computing for impact day 154
2023-07-13 19:50:37.360201 Computing for impact day 155
2023-07-13 19:54:18.530378 Computing for impact day 156
2023-07-13 19:57:59.249133 Computing for impact day 157
2023-07-13 20:01:40.416891 Computing for impact day 158
2023-07-13 20:05:21.330170 Computing for impact day 159
2023-07-13 20:09:03.770148 Computing for impact day 160
2023-07-13 20:12:46.285845 Computing for impact day 161
2023-07-13 20:16:28.143885 Computing for impact day 162
2023-07-13 20:20:10.330367 Computing for impact day 163
2023-07-13 20:23:51.061937 Computing for impact day 164
2023-07-13 20:27:33.337059 Computing for impact day 165
2023-07-13 20:31:14.898010 Computing for impact day 166
2023-07-13 20:34:54.686691 Computing for impact 

2023-07-14 04:32:46.858984 Computing for impact day 297
2023-07-14 04:36:26.965184 Computing for impact day 298
2023-07-14 04:40:07.362510 Computing for impact day 299
2023-07-14 04:43:47.297180 Computing for impact day 300
2023-07-14 04:47:28.840122 Computing for impact day 301
2023-07-14 04:51:08.803208 Computing for impact day 302
2023-07-14 04:54:49.824589 Computing for impact day 303
2023-07-14 04:58:30.522920 Computing for impact day 304
2023-07-14 05:02:10.702998 Computing for impact day 305
2023-07-14 05:05:49.966907 Computing for impact day 306
2023-07-14 05:09:30.024657 Computing for impact day 307
2023-07-14 05:13:10.398406 Computing for impact day 308
2023-07-14 05:16:50.965879 Computing for impact day 309
2023-07-14 05:20:31.158222 Computing for impact day 310
2023-07-14 05:24:10.594117 Computing for impact day 311
2023-07-14 05:27:49.911457 Computing for impact day 312
2023-07-14 05:31:29.348185 Computing for impact day 313
2023-07-14 05:35:09.353525 Computing for impact 

2023-07-14 13:34:27.305171 Computing for impact day 444
2023-07-14 13:38:08.105638 Computing for impact day 445
2023-07-14 13:41:49.197548 Computing for impact day 446
2023-07-14 13:45:28.641604 Computing for impact day 447
2023-07-14 13:49:08.599231 Computing for impact day 448
2023-07-14 13:52:48.404416 Computing for impact day 449
2023-07-14 13:56:27.490918 Computing for impact day 450
2023-07-14 14:00:07.815010 Computing for impact day 451
2023-07-14 14:03:47.321039 Computing for impact day 452
2023-07-14 14:07:28.586998 Computing for impact day 453
2023-07-14 14:11:08.002207 Computing for impact day 454
2023-07-14 14:14:49.175518 Computing for impact day 455
2023-07-14 14:18:28.659437 Computing for impact day 456
2023-07-14 14:22:08.648316 Computing for impact day 457
2023-07-14 14:25:47.584637 Computing for impact day 458
2023-07-14 14:29:28.639681 Computing for impact day 459
2023-07-14 14:33:07.647148 Computing for impact day 460
2023-07-14 14:36:47.707007 Computing for impact 

2023-07-14 22:32:52.908079 Computing for impact day 591
2023-07-14 22:36:31.966483 Computing for impact day 592
2023-07-14 22:40:10.387937 Computing for impact day 593
2023-07-14 22:43:48.769495 Computing for impact day 594
2023-07-14 22:47:27.709110 Computing for impact day 595
2023-07-14 22:51:06.564401 Computing for impact day 596
2023-07-14 22:54:46.532601 Computing for impact day 597
2023-07-14 22:58:25.307613 Computing for impact day 598
2023-07-14 23:02:05.453145 Computing for impact day 599
2023-07-14 23:05:44.473160 Computing for impact day 600
2023-07-14 23:09:23.742254 Computing for impact day 601
2023-07-14 23:13:03.448639 Computing for impact day 602
2023-07-14 23:16:42.322931 Computing for impact day 603
2023-07-14 23:20:22.322830 Computing for impact day 604
2023-07-14 23:24:01.749617 Computing for impact day 605
2023-07-14 23:27:41.998329 Computing for impact day 606
2023-07-14 23:31:20.607995 Computing for impact day 607
2023-07-14 23:35:01.170030 Computing for impact 

...

2023-07-18 23:40:23.761573 Computing for impact day 2183
2023-07-18 23:44:02.917418 Computing for impact day 2184
2023-07-18 23:47:42.555153 Computing for impact day 2185
2023-07-18 23:51:21.704453 Computing for impact day 2186
2023-07-18 23:55:01.111073 Computing for impact day 2187
2023-07-18 23:58:40.791789 Computing for impact day 2188
2023-07-19 00:02:20.513291 Computing for impact day 2189
2023-07-19 00:06:00.461693 Computing for impact day 2190
2023-07-19 00:09:41.270457 Computing for impact day 2191
2023-07-19 00:13:21.440255 Computing for impact day 2192
2023-07-19 00:17:02.090461 Computing for impact day 2193
2023-07-19 00:20:41.867101 Computing for impact day 2194
2023-07-19 00:24:22.417824 Computing for impact day 2195
2023-07-19 00:28:01.727629 Computing for impact day 2196
2023-07-19 00:31:41.534840 Computing for impact day 2197
2023-07-19 00:35:21.091150 Computing for impact day 2198
2023-07-19 00:39:02.508646 Computing for impact day 2199
2023-07-19 00:42:41.938912 Comp

2023-07-19 08:27:38.691613 Computing for impact day 2327
2023-07-19 08:31:18.213510 Computing for impact day 2328
2023-07-19 08:34:56.595180 Computing for impact day 2329
2023-07-19 08:38:36.229462 Computing for impact day 2330
2023-07-19 08:42:16.080270 Computing for impact day 2331
2023-07-19 08:45:54.373344 Computing for impact day 2332
2023-07-19 08:49:34.042645 Computing for impact day 2333
2023-07-19 08:53:14.521881 Computing for impact day 2334
2023-07-19 08:56:53.105137 Computing for impact day 2335
2023-07-19 09:00:32.860070 Computing for impact day 2336
2023-07-19 09:04:11.823944 Computing for impact day 2337
2023-07-19 09:07:52.150940 Computing for impact day 2338
2023-07-19 09:11:30.445426 Computing for impact day 2339
2023-07-19 09:15:10.376795 Computing for impact day 2340
2023-07-19 09:18:48.555371 Computing for impact day 2341
2023-07-19 09:22:28.131283 Computing for impact day 2342
2023-07-19 09:26:06.741295 Computing for impact day 2343
2023-07-19 09:29:47.201457 Comp

2023-07-19 17:15:25.257876 Computing for impact day 2471
2023-07-19 17:19:07.916587 Computing for impact day 2472
2023-07-19 17:22:50.942639 Computing for impact day 2473
2023-07-19 17:26:38.482744 Computing for impact day 2474
2023-07-19 17:30:18.836018 Computing for impact day 2475
2023-07-19 17:33:58.603759 Computing for impact day 2476
2023-07-19 17:37:58.765797 Computing for impact day 2477
2023-07-19 17:41:40.124897 Computing for impact day 2478
2023-07-19 17:45:20.258542 Computing for impact day 2479
2023-07-19 17:49:05.584288 Computing for impact day 2480
2023-07-19 17:52:45.982753 Computing for impact day 2481
2023-07-19 17:56:31.878606 Computing for impact day 2482
2023-07-19 18:00:10.249073 Computing for impact day 2483
2023-07-19 18:03:49.528410 Computing for impact day 2484
2023-07-19 18:07:27.332984 Computing for impact day 2485
2023-07-19 18:11:07.127823 Computing for impact day 2486
2023-07-19 18:14:45.140605 Computing for impact day 2487
2023-07-19 18:18:23.639624 Comp

2023-07-20 02:03:16.123727 Computing for impact day 2615
2023-07-20 02:06:57.318813 Computing for impact day 2616
2023-07-20 02:10:40.454888 Computing for impact day 2617
2023-07-20 02:14:22.730776 Computing for impact day 2618
2023-07-20 02:18:04.916989 Computing for impact day 2619
2023-07-20 02:21:47.077096 Computing for impact day 2620
2023-07-20 02:25:30.525161 Computing for impact day 2621
2023-07-20 02:29:12.836963 Computing for impact day 2622
2023-07-20 02:32:55.371599 Computing for impact day 2623
2023-07-20 02:36:38.185213 Computing for impact day 2624
2023-07-20 02:40:20.447395 Computing for impact day 2625
2023-07-20 02:44:02.710183 Computing for impact day 2626
2023-07-20 02:47:44.902902 Computing for impact day 2627
2023-07-20 02:51:26.991653 Computing for impact day 2628
2023-07-20 02:55:08.448106 Computing for impact day 2629
2023-07-20 02:58:50.620668 Computing for impact day 2630
2023-07-20 03:02:32.509247 Computing for impact day 2631
2023-07-20 03:06:14.915835 Comp

2023-07-20 10:53:14.497595 Computing for impact day 2759
2023-07-20 10:56:53.831661 Computing for impact day 2760
2023-07-20 11:00:33.656512 Computing for impact day 2761
2023-07-20 11:04:14.165253 Computing for impact day 2762
2023-07-20 11:07:54.288908 Computing for impact day 2763
2023-07-20 11:11:34.601963 Computing for impact day 2764
2023-07-20 11:15:14.440007 Computing for impact day 2765
2023-07-20 11:18:53.843018 Computing for impact day 2766
2023-07-20 11:22:33.482471 Computing for impact day 2767
2023-07-20 11:26:12.946840 Computing for impact day 2768
2023-07-20 11:29:52.752129 Computing for impact day 2769
2023-07-20 11:33:31.722672 Computing for impact day 2770


## Aggregate impact day level metrics

In [6]:
time_delta = 30
    
datasets_directory = '/data/tmp/smehra/aggregated_data/poppy/visits_per_district_day/time_delta_' + str(time_delta) + '_days/impact_day_level/'

for impact_day in range(1, 2771):
    
    print(str(datetime.now()) + ' Computing for impact day ' + str(impact_day))
    impact_day_folder = datasets_directory + 'impact_day_' + str(impact_day)
    
    # read impact_day level dataset
    impact_day_data = ddf.read_csv(impact_day_folder + '/*',
                                         dtype = {'origin_district': 'int16',
                                                  'impact_day': 'int16',
                                                  'impacted': 'int32',
                                                  'destination_district': 'int16',
                                                  'visit_day': 'int16',
                                                  'visits': 'float64'})

    # calculate sum of "impacted" users for each group
    impacted_agg = impact_day_data.groupby(['origin_district', 'impact_day', 'destination_district', 'visit_day'])['impacted'].sum().reset_index()

    # drop rows where impacted_users count were zero i.e visits were NA
    visits_agg = impact_day_data[~impact_day_data.visits.isna()]
    # calculate sum of "visits" for each group
    visits_agg = visits_agg.groupby(['origin_district', 'impact_day', 'destination_district', 'visit_day'])['visits'].sum().reset_index()
    
    # merge to have total impacted and visits count in same dataset
    impacted_and_visits_merged = impacted_agg.merge(visits_agg, on = ['origin_district', 'impact_day', 'destination_district', 'visit_day'], how = 'left')
    
    impacted_and_visits_merged.to_csv('/data/tmp/smehra/aggregated_data/poppy/visits_per_district_day/time_delta_' + str(time_delta) + '_days/aggregated/impact_day_' + str(impact_day) + '.csv', index = False, single_file = True)


2023-07-21 09:30:18.083746 Computing for impact day 1
2023-07-21 09:30:58.466989 Computing for impact day 2
2023-07-21 09:31:39.191213 Computing for impact day 3
2023-07-21 09:32:19.715203 Computing for impact day 4
2023-07-21 09:33:00.361027 Computing for impact day 5
2023-07-21 09:33:41.567416 Computing for impact day 6
2023-07-21 09:36:31.955129 Computing for impact day 10
2023-07-21 09:37:14.789245 Computing for impact day 11
2023-07-21 09:37:59.333161 Computing for impact day 12
2023-07-21 09:38:43.657559 Computing for impact day 13
2023-07-21 09:39:26.530970 Computing for impact day 14
2023-07-21 09:40:10.274011 Computing for impact day 15
2023-07-21 09:40:53.254787 Computing for impact day 16
2023-07-21 09:41:37.871230 Computing for impact day 17
2023-07-21 09:42:21.880435 Computing for impact day 18
2023-07-21 09:43:05.759537 Computing for impact day 19
2023-07-21 09:43:49.822934 Computing for impact day 20
2023-07-21 09:44:34.883063 Computing for impact day 21
2023-07-21 09:45

2023-07-21 11:28:48.295553 Computing for impact day 153
2023-07-21 11:29:37.023220 Computing for impact day 154
2023-07-21 11:30:27.634069 Computing for impact day 155
2023-07-21 11:31:17.815662 Computing for impact day 156
2023-07-21 11:32:07.247586 Computing for impact day 157
2023-07-21 11:32:57.926273 Computing for impact day 158
2023-07-21 11:33:51.085399 Computing for impact day 159
2023-07-21 11:34:40.333025 Computing for impact day 160
2023-07-21 11:35:29.036346 Computing for impact day 161
2023-07-21 11:36:17.538221 Computing for impact day 162
2023-07-21 11:37:05.996570 Computing for impact day 163
2023-07-21 11:37:55.539231 Computing for impact day 164
2023-07-21 11:38:45.481671 Computing for impact day 165
2023-07-21 11:39:36.310215 Computing for impact day 166
2023-07-21 11:40:25.745341 Computing for impact day 167
2023-07-21 11:41:15.529445 Computing for impact day 168
2023-07-21 11:42:06.360956 Computing for impact day 169
2023-07-21 11:42:54.679084 Computing for impact 

2023-07-21 13:32:59.755211 Computing for impact day 300
2023-07-21 13:33:50.801936 Computing for impact day 301
2023-07-21 13:34:43.745381 Computing for impact day 302
2023-07-21 13:35:35.255848 Computing for impact day 303
2023-07-21 13:36:27.046088 Computing for impact day 304
2023-07-21 13:37:18.988163 Computing for impact day 305
2023-07-21 13:38:10.762327 Computing for impact day 306
2023-07-21 13:39:04.090270 Computing for impact day 307
2023-07-21 13:39:54.881882 Computing for impact day 308
2023-07-21 13:40:46.064409 Computing for impact day 309
2023-07-21 13:41:35.699100 Computing for impact day 310
2023-07-21 13:42:26.320718 Computing for impact day 311
2023-07-21 13:43:18.068348 Computing for impact day 312
2023-07-21 13:44:11.163500 Computing for impact day 313
2023-07-21 13:45:02.350907 Computing for impact day 314
2023-07-21 13:45:54.904877 Computing for impact day 315
2023-07-21 13:46:47.063189 Computing for impact day 316
2023-07-21 13:47:37.594634 Computing for impact 

2023-07-21 15:39:51.768692 Computing for impact day 447
2023-07-21 15:40:43.710153 Computing for impact day 448
2023-07-21 15:41:36.074244 Computing for impact day 449
2023-07-21 15:42:25.910874 Computing for impact day 450
2023-07-21 15:43:18.680664 Computing for impact day 451
2023-07-21 15:44:11.574990 Computing for impact day 452
2023-07-21 15:45:01.520223 Computing for impact day 453
2023-07-21 15:45:54.158309 Computing for impact day 454
2023-07-21 15:46:46.859866 Computing for impact day 455
2023-07-21 15:47:38.065588 Computing for impact day 456
2023-07-21 15:48:32.650581 Computing for impact day 457
2023-07-21 15:49:23.961022 Computing for impact day 458
2023-07-21 15:50:15.076525 Computing for impact day 459
2023-07-21 15:51:06.705695 Computing for impact day 460
2023-07-21 15:52:00.440034 Computing for impact day 461
2023-07-21 15:52:50.932207 Computing for impact day 462
2023-07-21 15:53:43.523624 Computing for impact day 463
2023-07-21 15:54:34.580724 Computing for impact 

2023-07-21 17:47:20.657961 Computing for impact day 594
2023-07-21 17:48:12.834195 Computing for impact day 595
2023-07-21 17:49:08.667295 Computing for impact day 596
2023-07-21 17:49:59.224610 Computing for impact day 597
2023-07-21 17:50:52.301242 Computing for impact day 598
2023-07-21 17:51:43.832602 Computing for impact day 599
2023-07-21 17:52:38.116590 Computing for impact day 600
2023-07-21 17:53:29.802672 Computing for impact day 601
2023-07-21 17:54:22.809375 Computing for impact day 602
2023-07-21 17:55:15.191376 Computing for impact day 603
2023-07-21 17:56:08.102962 Computing for impact day 604
2023-07-21 17:57:01.859932 Computing for impact day 605
2023-07-21 17:57:52.819609 Computing for impact day 606
2023-07-21 17:58:43.228848 Computing for impact day 607
2023-07-21 17:59:34.968062 Computing for impact day 608
2023-07-21 18:00:29.419672 Computing for impact day 609
2023-07-21 18:01:23.730106 Computing for impact day 610
2023-07-21 18:02:18.182250 Computing for impact 

...

2023-07-22 16:39:56.513783 Computing for impact day 2186
2023-07-22 16:40:47.595556 Computing for impact day 2187
2023-07-22 16:41:42.662498 Computing for impact day 2188
2023-07-22 16:42:33.831161 Computing for impact day 2189
2023-07-22 16:43:24.883130 Computing for impact day 2190
2023-07-22 16:44:16.116938 Computing for impact day 2191
2023-07-22 16:45:08.037754 Computing for impact day 2192
2023-07-22 16:45:56.933464 Computing for impact day 2193
2023-07-22 16:46:47.318584 Computing for impact day 2194
2023-07-22 16:47:40.597508 Computing for impact day 2195
2023-07-22 16:48:31.314583 Computing for impact day 2196
2023-07-22 16:49:22.189592 Computing for impact day 2197
2023-07-22 16:50:15.676735 Computing for impact day 2198
2023-07-22 16:51:07.633704 Computing for impact day 2199
2023-07-22 16:51:59.191841 Computing for impact day 2200
2023-07-22 16:52:49.631778 Computing for impact day 2201
2023-07-22 16:53:39.772948 Computing for impact day 2202
2023-07-22 16:54:32.384659 Comp

2023-07-22 18:43:20.051994 Computing for impact day 2330
2023-07-22 18:44:09.746691 Computing for impact day 2331
2023-07-22 18:44:59.717057 Computing for impact day 2332
2023-07-22 18:45:50.234159 Computing for impact day 2333
2023-07-22 18:46:41.736354 Computing for impact day 2334
2023-07-22 18:47:31.673504 Computing for impact day 2335
2023-07-22 18:48:24.193850 Computing for impact day 2336
2023-07-22 18:49:16.650141 Computing for impact day 2337
2023-07-22 18:50:07.866348 Computing for impact day 2338
2023-07-22 18:50:57.220139 Computing for impact day 2339
2023-07-22 18:51:45.876490 Computing for impact day 2340
2023-07-22 18:52:35.955910 Computing for impact day 2341
2023-07-22 18:53:25.967310 Computing for impact day 2342
2023-07-22 18:54:16.533295 Computing for impact day 2343
2023-07-22 18:55:09.489674 Computing for impact day 2344
2023-07-22 18:56:00.613502 Computing for impact day 2345
2023-07-22 18:56:51.961608 Computing for impact day 2346
2023-07-22 18:57:43.226978 Comp

2023-07-22 20:46:23.092136 Computing for impact day 2474
2023-07-22 20:47:13.167254 Computing for impact day 2475
2023-07-22 20:48:04.910712 Computing for impact day 2476
2023-07-22 20:49:00.738066 Computing for impact day 2477
2023-07-22 20:49:49.990194 Computing for impact day 2478
2023-07-22 20:50:42.280384 Computing for impact day 2479
2023-07-22 20:51:32.698261 Computing for impact day 2480
2023-07-22 20:52:23.847593 Computing for impact day 2481
2023-07-22 20:53:15.890723 Computing for impact day 2482
2023-07-22 20:54:06.551286 Computing for impact day 2483
2023-07-22 20:54:55.712678 Computing for impact day 2484
2023-07-22 20:55:47.037592 Computing for impact day 2485
2023-07-22 20:56:37.155481 Computing for impact day 2486
2023-07-22 20:57:27.502559 Computing for impact day 2487
2023-07-22 20:58:18.286425 Computing for impact day 2488
2023-07-22 20:59:09.488484 Computing for impact day 2489
2023-07-22 20:59:59.509831 Computing for impact day 2490
2023-07-22 21:00:52.509171 Comp

2023-07-22 22:49:38.080329 Computing for impact day 2618
2023-07-22 22:50:28.078674 Computing for impact day 2619
2023-07-22 22:51:20.047215 Computing for impact day 2620
2023-07-22 22:52:11.046863 Computing for impact day 2621
2023-07-22 22:53:03.187482 Computing for impact day 2622
2023-07-22 22:53:55.092128 Computing for impact day 2623
2023-07-22 22:54:46.287197 Computing for impact day 2624
2023-07-22 22:55:36.836387 Computing for impact day 2625
2023-07-22 22:56:27.637773 Computing for impact day 2626
2023-07-22 22:57:18.377539 Computing for impact day 2627
2023-07-22 22:58:09.287480 Computing for impact day 2628
2023-07-22 22:59:00.708601 Computing for impact day 2629
2023-07-22 22:59:51.459125 Computing for impact day 2630
2023-07-22 23:00:41.700100 Computing for impact day 2631
2023-07-22 23:01:32.229184 Computing for impact day 2632
2023-07-22 23:02:21.711065 Computing for impact day 2633
2023-07-22 23:03:12.224906 Computing for impact day 2634
2023-07-22 23:04:03.910415 Comp

2023-07-23 00:50:17.298388 Computing for impact day 2762
2023-07-23 00:51:00.455198 Computing for impact day 2763
2023-07-23 00:51:44.457597 Computing for impact day 2764
2023-07-23 00:52:28.168516 Computing for impact day 2765
2023-07-23 00:53:12.676100 Computing for impact day 2766
2023-07-23 00:53:57.395044 Computing for impact day 2767
2023-07-23 00:54:41.785200 Computing for impact day 2768
2023-07-23 00:55:26.287965 Computing for impact day 2769
2023-07-23 00:56:11.398120 Computing for impact day 2770
